In [1]:
import numpy as np
import pandas as pd

import os

In [3]:
train_data_path = '../automl-fall-school-2021-hackathon/data_train.csv'
test_data_path = '../automl-fall-school-2021-hackathon/data_test.csv'

In [4]:
train_data = pd.read_csv(train_data_path) # consists of 50000 rows and 100 features 
# (V25_6, V26_5, V47_8, V52_6, V56_7,v69_5, V74_4,V83_5, V99_7 and V100_6 are categorical)
    
test_data = pd.read_csv(test_data_path) # consists of 66238 rows and 100 features
# last column is dummy column)

In [5]:
# Dropping useless columns from the training data
train_data = train_data.drop(['V5', 'V23', 'V34', 'V44', 'V66', 'V86' ], axis=1)

# Dropping all rows with any nan values
train_data = train_data.dropna()

In [6]:
train_data

,class,V1,V2,V3,V4,V6,V7,V8,V9,V10,...,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100
0,DO,1819.870220,-1049.683290,-1097.551530,419.668781,874.102780,-5646.018630,-1745.922933,1257.986598,876.953151,...,907.050893,118.392812,3723.116140,570.075438,-833.691337,2797.442697,306.103167,863.381036,D,D
1,AT,5338.339092,-896.336093,-169.203551,420.113222,875.912844,990.045875,-490.424433,-315.042299,871.569241,...,930.631453,144.072375,4081.401627,447.463827,-722.890871,-9411.034432,265.863035,449.478649,B,D
2,DZ,376.999942,-457.473122,-12.817029,420.887318,875.878783,-290.534515,-275.232733,-373.929651,878.186110,...,940.574847,144.645611,133.532673,329.363657,-760.617723,1389.481995,303.399314,417.676998,D,D
3,EF,6633.434350,-1139.578274,562.928815,422.742834,869.440917,1062.673252,-1296.822459,-638.085049,878.147031,...,940.457668,141.576882,-840.502401,439.238111,-763.469013,-4605.075778,316.002515,502.876430,D,D
4,DW,-3147.858218,-1315.076756,-1640.884995,422.072351,869.445753,-2577.051356,-693.467931,-41.339765,881.863079,...,794.539528,146.561762,1052.697529,51.096985,-781.918944,-4423.057820,269.091842,377.266730,D,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,CK,637.475113,-509.604945,68.401386,420.166775,875.272112,1531.888185,-224.819844,-390.949253,879.640508,...,1001.366049,138.985893,-23.071407,397.246282,-762.546047,-138.500465,272.495833,433.128325,D,D
49996,EH,4138.165843,-677.186860,-221.581569,420.679631,880.584017,548.687942,33.667961,-30.767771,877.663037,...,1083.243078,145.314090,-164.709859,47.029279,-788.976170,5319.476900,297.987579,503.791602,E,D
49997,CW,-6576.137973,-336.252493,-259.617310,420.281572,875.880073,-2187.145059,-492.867469,-735.107623,881.855471,...,897.097722,145.853874,1025.685762,297.677559,-809.616688,4874.145139,363.358205,439.075537,C,D
49998,BW,-16736.671310,-334.539742,-521.619908,421.936298,875.470550,-1832.102252,-1588.345249,-157.507237,884.924972,...,833.384112,120.439889,71.780507,271.855923,-826.082828,-970.042171,339.248619,416.835122,E,C


In [14]:
def convertCategoricalsToNumerics(df):
    
    categorical_fields = df.select_dtypes(include=['object'])
    categorical_fields = categorical_fields.drop('class',axis=1)
    
    for col in df.columns:
        if col in categorical_fields.columns:
                       
            # Counting the different categories in the column
            countOrdered_cats = list(dict(categorical_fields[col].value_counts()).keys()) # descending order
            
            
            # Creating a list of numeric replacements
            num_replacements = list(np.arange(len(countOrdered_cats))+1)
            
            # Dictionary of replacements to pass to df.replace()
            replacements = dict(zip(countOrdered_cats, num_replacements))
    
            cd = df[col].replace(replacements, inplace=True)
        
        else:
            pass
        
    return df
    
train_numData = convertCategoricalsToNumerics(train_data)

## Buidling ANN using Keras

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils


In [16]:
X = train_numData.loc[:, train_numData.columns != 'class']

Y = train_numData.loc[:, train_numData.columns == 'class']

In [44]:
test_size = 0.20
shuffle = True

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=test_size, shuffle=shuffle)

In [50]:
np.shape(train_X.values)

(39977, 94)

In [31]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(train_Y['class'])
encoded_Y = encoder.transform(train_Y['class'])

In [54]:
len(encoded_Y)

49972

In [35]:
dummy_y = np_utils.to_categorical(encoded_Y)

In [42]:
dummy_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [55]:
# encode 'class' values as integers
encoder = LabelEncoder()
encoder.fit(train_Y['class'])
encoded_Y = encoder.transform(train_Y['class'])

# convert integers to dummy variables (i.e. one hot encoded)
onehot_Y = np_utils.to_categorical(encoded_Y)

In [56]:
# define baseline model
def baseline_model():
    model = Sequential()
    model.add(Dense(512, input_dim=94, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(100, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 

In [57]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=1024, verbose=1)

results = cross_val_score(estimator, train_X.values, onehot_Y, cv=KFold(n_splits=5, shuffle=True))

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

2021-11-11 01:53:06.831459: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-11 01:53:06.941141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 01:53:06.941736: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1050 computeCapability: 6.1
coreClock: 1.493GHz coreCount: 5 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 104.43GiB/s
2021-11-11 01:53:06.942026: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-11 01:53:06.942237: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libc

Epoch 1/200
32/32 [==============================] - 29s 38ms/step - loss: 89061127.3939 - accuracy: 0.0125
Epoch 2/200
32/32 [==============================] - 1s 39ms/step - loss: 588628.4413 - accuracy: 0.0127
Epoch 3/200
32/32 [==============================] - 1s 37ms/step - loss: 32.7058 - accuracy: 0.0170
Epoch 4/200
32/32 [==============================] - 1s 39ms/step - loss: 10.9863 - accuracy: 0.0169
Epoch 5/200
32/32 [==============================] - 1s 46ms/step - loss: 9.8801 - accuracy: 0.0191
Epoch 6/200
32/32 [==============================] - 1s 39ms/step - loss: 5.5709 - accuracy: 0.0201
Epoch 7/200
32/32 [==============================] - 1s 46ms/step - loss: 5.2148 - accuracy: 0.0206
Epoch 8/200
32/32 [==============================] - 1s 37ms/step - loss: 8.5834 - accuracy: 0.0216
Epoch 9/200
32/32 [==============================] - 1s 41ms/step - loss: 5.3681 - accuracy: 0.0216
Epoch 10/200
32/32 [==============================] - 1s 41ms/step - loss: 5.0392 - a

32/32 [==============================] - 1s 38ms/step - loss: 4.5333 - accuracy: 0.0250
Epoch 83/200
32/32 [==============================] - 1s 39ms/step - loss: 4.5382 - accuracy: 0.0278
Epoch 84/200
32/32 [==============================] - 1s 30ms/step - loss: 4.5571 - accuracy: 0.0247
Epoch 85/200
32/32 [==============================] - 1s 37ms/step - loss: 4.5345 - accuracy: 0.0264
Epoch 86/200
32/32 [==============================] - 1s 42ms/step - loss: 4.5304 - accuracy: 0.0273
Epoch 87/200
32/32 [==============================] - 1s 36ms/step - loss: 4.5298 - accuracy: 0.0267
Epoch 88/200
32/32 [==============================] - 1s 41ms/step - loss: 4.5284 - accuracy: 0.0269
Epoch 89/200
32/32 [==============================] - 1s 42ms/step - loss: 4.5264 - accuracy: 0.0264
Epoch 90/200
32/32 [==============================] - 1s 35ms/step - loss: 4.5224 - accuracy: 0.0273
Epoch 91/200
32/32 [==============================] - 1s 40ms/step - loss: 4.5285 - accuracy: 0.0270
Epo

32/32 [==============================] - 1s 42ms/step - loss: 4.1515 - accuracy: 0.0500
Epoch 163/200
32/32 [==============================] - 1s 39ms/step - loss: 4.1363 - accuracy: 0.0473
Epoch 164/200
32/32 [==============================] - 1s 39ms/step - loss: 4.1479 - accuracy: 0.0460
Epoch 165/200
32/32 [==============================] - 1s 42ms/step - loss: 4.1343 - accuracy: 0.0510
Epoch 166/200
32/32 [==============================] - 1s 37ms/step - loss: 4.1291 - accuracy: 0.0454
Epoch 167/200
32/32 [==============================] - 1s 33ms/step - loss: 4.1327 - accuracy: 0.0468
Epoch 168/200
32/32 [==============================] - 1s 40ms/step - loss: 4.1323 - accuracy: 0.0531
Epoch 169/200
32/32 [==============================] - 1s 42ms/step - loss: 4.1515 - accuracy: 0.0482
Epoch 170/200
32/32 [==============================] - 1s 42ms/step - loss: 4.1253 - accuracy: 0.0513
Epoch 171/200
32/32 [==============================] - 1s 41ms/step - loss: 4.1614 - accuracy: 0

32/32 [==============================] - 1s 43ms/step - loss: 4.5669 - accuracy: 0.0216
Epoch 42/200
32/32 [==============================] - 1s 39ms/step - loss: 4.5675 - accuracy: 0.0221
Epoch 43/200
32/32 [==============================] - 1s 42ms/step - loss: 4.5682 - accuracy: 0.0208
Epoch 44/200
32/32 [==============================] - 1s 45ms/step - loss: 4.5675 - accuracy: 0.0227
Epoch 45/200
32/32 [==============================] - 2s 49ms/step - loss: 4.5695 - accuracy: 0.0211
Epoch 46/200
32/32 [==============================] - 2s 51ms/step - loss: 4.5666 - accuracy: 0.0217
Epoch 47/200
32/32 [==============================] - 2s 51ms/step - loss: 4.5680 - accuracy: 0.0204
Epoch 48/200
32/32 [==============================] - 2s 51ms/step - loss: 4.5654 - accuracy: 0.0201
Epoch 49/200
32/32 [==============================] - 1s 46ms/step - loss: 4.5655 - accuracy: 0.0222
Epoch 50/200
32/32 [==============================] - 2s 53ms/step - loss: 4.5671 - accuracy: 0.0206
Epo

...

32/32 [==============================] - 1s 46ms/step - loss: 4.5623 - accuracy: 0.0215
Epoch 41/200
32/32 [==============================] - 1s 42ms/step - loss: 4.5621 - accuracy: 0.0213
Epoch 42/200
32/32 [==============================] - 2s 48ms/step - loss: 4.5663 - accuracy: 0.0223
Epoch 43/200
32/32 [==============================] - 1s 44ms/step - loss: 4.5646 - accuracy: 0.0229
Epoch 44/200
32/32 [==============================] - 1s 44ms/step - loss: 4.5644 - accuracy: 0.0233
Epoch 45/200
32/32 [==============================] - 1s 41ms/step - loss: 4.5583 - accuracy: 0.0250
Epoch 46/200
32/32 [==============================] - 1s 43ms/step - loss: 4.5612 - accuracy: 0.0230
Epoch 47/200
32/32 [==============================] - 1s 46ms/step - loss: 4.5634 - accuracy: 0.0224
Epoch 48/200
32/32 [==============================] - 2s 49ms/step - loss: 4.5591 - accuracy: 0.0243
Epoch 49/200
32/32 [==============================] - 2s 55ms/step - loss: 4.5619 - accuracy: 0.0221
Epo

32/32 [==============================] - 1s 40ms/step - loss: 4.5609 - accuracy: 0.0222
Epoch 121/200
32/32 [==============================] - 1s 38ms/step - loss: 4.5629 - accuracy: 0.0207
Epoch 122/200
32/32 [==============================] - 1s 40ms/step - loss: 4.5656 - accuracy: 0.0214
Epoch 123/200
32/32 [==============================] - 1s 39ms/step - loss: 4.5649 - accuracy: 0.0205
Epoch 124/200
32/32 [==============================] - 1s 45ms/step - loss: 4.5666 - accuracy: 0.0218
Epoch 125/200
32/32 [==============================] - 1s 40ms/step - loss: 4.5619 - accuracy: 0.0217
Epoch 126/200
32/32 [==============================] - 1s 38ms/step - loss: 4.5640 - accuracy: 0.0201
Epoch 127/200
32/32 [==============================] - 1s 41ms/step - loss: 4.5645 - accuracy: 0.0223
Epoch 128/200
32/32 [==============================] - 1s 36ms/step - loss: 4.5661 - accuracy: 0.0212
Epoch 129/200
32/32 [==============================] - 1s 42ms/step - loss: 4.5650 - accuracy: 0

8/8 [==============================] - 1s 14ms/step - loss: 4.5743 - accuracy: 0.0176
Epoch 1/200
32/32 [==============================] - 3s 40ms/step - loss: 127348894.7879 - accuracy: 0.0101
Epoch 2/200
32/32 [==============================] - 1s 34ms/step - loss: 19062349.3030 - accuracy: 0.0111
Epoch 3/200
32/32 [==============================] - 1s 42ms/step - loss: 3515050.8030 - accuracy: 0.0181
Epoch 4/200
32/32 [==============================] - 2s 49ms/step - loss: 931282.7472 - accuracy: 0.0192
Epoch 5/200
32/32 [==============================] - 1s 40ms/step - loss: 326.1629 - accuracy: 0.0233
Epoch 6/200
32/32 [==============================] - 1s 38ms/step - loss: 12048.0524 - accuracy: 0.0238
Epoch 7/200
32/32 [==============================] - 1s 38ms/step - loss: 12.5307 - accuracy: 0.0195
Epoch 8/200
32/32 [==============================] - 2s 49ms/step - loss: 7.3482 - accuracy: 0.0168
Epoch 9/200
32/32 [==============================] - 1s 41ms/step - loss: 7.2693 

32/32 [==============================] - 1s 44ms/step - loss: 4.1761 - accuracy: 0.0436
Epoch 81/200
32/32 [==============================] - 1s 38ms/step - loss: 4.1754 - accuracy: 0.0453
Epoch 82/200
32/32 [==============================] - 1s 43ms/step - loss: 4.1685 - accuracy: 0.0445
Epoch 83/200
32/32 [==============================] - 1s 33ms/step - loss: 4.1728 - accuracy: 0.0433
Epoch 84/200
32/32 [==============================] - 1s 41ms/step - loss: 4.1622 - accuracy: 0.0466
Epoch 85/200
32/32 [==============================] - 1s 41ms/step - loss: 4.1556 - accuracy: 0.0466
Epoch 86/200
32/32 [==============================] - 1s 40ms/step - loss: 4.1456 - accuracy: 0.0480
Epoch 87/200
32/32 [==============================] - 1s 40ms/step - loss: 4.1365 - accuracy: 0.0506
Epoch 88/200
32/32 [==============================] - 1s 39ms/step - loss: 4.1366 - accuracy: 0.0471
Epoch 89/200
32/32 [==============================] - 2s 49ms/step - loss: 4.1303 - accuracy: 0.0494
Epo

32/32 [==============================] - 1s 40ms/step - loss: 4.1228 - accuracy: 0.0431
Epoch 161/200
32/32 [==============================] - 1s 34ms/step - loss: 4.1129 - accuracy: 0.0511
Epoch 162/200
32/32 [==============================] - 1s 38ms/step - loss: 4.1377 - accuracy: 0.0448
Epoch 163/200
32/32 [==============================] - 1s 40ms/step - loss: 4.0843 - accuracy: 0.0466
Epoch 164/200
32/32 [==============================] - 1s 40ms/step - loss: 4.0568 - accuracy: 0.0550
Epoch 165/200
32/32 [==============================] - 1s 39ms/step - loss: 4.0498 - accuracy: 0.0541
Epoch 166/200
32/32 [==============================] - 1s 41ms/step - loss: 4.0713 - accuracy: 0.0520
Epoch 167/200
32/32 [==============================] - 1s 43ms/step - loss: 4.0462 - accuracy: 0.0533
Epoch 168/200
32/32 [==============================] - 1s 43ms/step - loss: 4.0734 - accuracy: 0.0533
Epoch 169/200
32/32 [==============================] - 2s 47ms/step - loss: 4.0270 - accuracy: 0